In [1]:
! pip3 install -q findspark
import findspark
findspark.init()

In [2]:
import pyspark
from pyspark.sql import SparkSession

In [3]:
spark=SparkSession.builder.master("local[*]").appName("clustername").getOrCreate()
spark

In [4]:
df=spark.read.format("csv").load("CompleteDataset.csv",inferSchema=True,header=True)
df.show(1)

+---+-----------------+---+--------------------+-----------+--------------------+-------+---------+--------------+--------------------+------+-----+-------+------------+----------+-------+-------+------------+---------+--------+-----+---------+---------+------------------+---------+-----------+----------+--------------+-----------+----------------+-------------+-------+------------+----------+-------+---------+-----------+---------+-------------+----------+--------------+------------+-------+---------------+--------+------+-------+----+----+----+----+----+-----+----+----+----+----+----+----+----+----+----+----+-------------------+----+----+----+----+----+----+----+----+----+----+----+
|_c0|             Name|Age|               Photo|Nationality|                Flag|Overall|Potential|          Club|           Club Logo| Value| Wage|Special|Acceleration|Aggression|Agility|Balance|Ball control|Composure|Crossing|Curve|Dribbling|Finishing|Free kick accuracy|GK diving|GK handling|GK kicking|

In [5]:
# How many partitions ?
print(df.rdd.getNumPartitions())
# Note
# Here we need to convert to RDD from data frame to get the num of partitions

2


In [6]:
# Increases or Dcreases the partition in DF
df2 = df.repartition(4)
df2.rdd.getNumPartitions()

4

In [7]:
#Show Df
# df2.show(1)

In [8]:
# Rename Columns and 
df2.withColumnRenamed("_c0","Id") \
    .withColumnRenamed("Ball Control","Ball_Control")
df2.show()

+----+---------------+---+--------------------+-----------+--------------------+-------+---------+--------------------+--------------------+------+-----+-------+------------+----------+-------+-------+------------+---------+--------+-----+---------+---------+------------------+---------+-----------+----------+--------------+-----------+----------------+-------------+-------+------------+----------+-------+---------+-----------+---------+-------------+----------+--------------+------------+-------+---------------+--------+------+-------+----+----+----+----+----+------+----+----+----+----+----+----+----+----+----+----+-------------------+----+----+----+----+----+----+----+----+----+----+----+
| _c0|           Name|Age|               Photo|Nationality|                Flag|Overall|Potential|                Club|           Club Logo| Value| Wage|Special|Acceleration|Aggression|Agility|Balance|Ball control|Composure|Crossing|Curve|Dribbling|Finishing|Free kick accuracy|GK diving|GK handling|

### Transformation

### Advanced DF Operations : Spark SQL and UDF

### Spark SQL

In [9]:
df2.createOrReplaceTempView("df_football")

In [10]:
sql_query="SELECT Age,count(*) as Count From df_football WHERE Overall > 70 GROUP BY Age ORDER BY Age"
result=spark.sql(sql_query)
result.show()

+---+-----+
|Age|Count|
+---+-----+
| 17|    1|
| 18|   13|
| 19|   41|
| 20|  105|
| 21|  160|
| 22|  246|
| 23|  296|
| 24|  369|
| 25|  486|
| 26|  418|
| 27|  444|
| 28|  395|
| 29|  474|
| 30|  359|
| 31|  272|
| 32|  206|
| 33|  283|
| 34|  101|
| 35|   74|
| 36|   57|
+---+-----+
only showing top 20 rows



### UDF

In [13]:
def uppercase_converter(record):
    if len(record) > 10 :
        return record.upper()
    else:
        return record.lower()

#register the DF
df2.createOrReplaceTempView('UDF_football')

#register the function
spark.udf.register("UPPER",uppercase_converter)

# use the UDF in SQL
sql_query="SELECT Age,UPPER(NAME) as Name,UPPER(Club) as Club FROM UDF_football"

result=spark.sql(sql_query)
result.show()

+---+----------------+--------------------+
|Age|            Name|                Club|
+---+----------------+--------------------+
| 19|    DOUGLAS LUIZ|           girona cf|
| 23|       a. schöpf|       FC SCHALKE 04|
| 27|      r. eckardt|  FC CARL ZEISS JENA|
| 27|     RYAN MENDES|         KAYSERISPOR|
| 26|    H. YAMAGUCHI|        CEREZO OSAKA|
| 29|   D. MCGOLDRICK|        IPSWICH TOWN|
| 25|            iago|       MOREIRENSE FC|
| 30| G. SCOGNAMIGLIO|              cesena|
| 31|           galán|      RAYO VALLECANO|
| 25|      p. wszołek| QUEENS PARK RANGERS|
| 22|         d. kort|      POGOŃ SZCZECIN|
| 23|      g. caprari|           sampdoria|
| 26|    KIKO FEMENÍA|             watford|
| 23|ALEXANDRE GUEDES|             cd aves|
| 35|   Y. KERMORGANT|             reading|
| 33|      g. hormaza|CLUB DE DEPORTES ...|
| 20|         j. boga|     BIRMINGHAM CITY|
| 20|    AURÉLIO BUTA|    ROYAL ANTWERP FC|
| 25|        y. komai|  URAWA RED DIAMONDS|
| 34|       a. rochat|  FC LAUSA